Diego Toribio <br>
Professor Sam Keene <br>
Frequentist Machine Learning <br>
Project 3: Model Assessment and Validation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import scipy.sparse.linalg
from sklearn.neighbors import KNeighborsClassifier as KNN

Data Prep

In [ ]:
np.random.seed(13)

# Parameters
N = 50        # Samples
p = 5000      # Original predictors
K = 6         # Folds
H = 100       # Best predictors
nn = 1        # 1-nearest neighbors

# Dataset
X = np.random.randn(N, p)
y = np.random.rand(1, N)
Y = np.where(y > 0.5, 1, 0).flatten()

print(X.shape, Y.shape)

(50, 5000) (50,)


Wrong Way:

This is the wrong way because

In [ ]:
# Reduce Data Points to Best Predictors
corr_xy = Y @ X
best = np.argsort(corr_xy)[-H:][::-1]
X_best = X[:, best]

print(X_best.shape)

(50, 100)


In [ ]:
corr_xy = Y @ X

best = np.argsort(corr_xy)[-H:][::-1]

num_samples_per_fold = N // K
errors = []

for i in range(K):
    # Calculate the start and end indices for the current fold
    start_val = i * num_samples_per_fold
    end_val = start_val + num_samples_per_fold
    if i == K-1:
        end_val = N

    # Split Data
    X_val = X_best[start_val:end_val]
    Y_val = Y[start_val:end_val]
    X_train = np.concatenate([X_best[:start_val], X_best[end_val:]])
    Y_train = np.concatenate([Y[:start_val], Y[end_val:]])

    # Train
    knn = KNN(n_neighbors=1)
    knn.fit(X_train, Y_train)

    # Validate
    accuracy = knn.score(X_val, Y_val)
    error = 1 - accuracy
    errors.append(error)

# Calculate the average percent error
average_percent_error = np.mean(errors) * 100
print("Average percent error: {:.2f}%".format(average_percent_error))

Average percent error: 9.17%


Right Way: \\

This is the right way because

In [ ]:
Num_samp = int(N / K)
correct = []

for i in range(K):
    # Split Data
    X_val = X[i * Num_samp:(i + 1) * Num_samp, :]
    Y_val = Y[i * Num_samp:(i + 1) * Num_samp]
    X_train = np.concatenate([X[:i * Num_samp, :], X[(i + 1) * Num_samp:, :]], axis=0)
    Y_train = np.concatenate([Y[:i * Num_samp], Y[(i + 1) * Num_samp:]])

    # Compute raw scores using matrix multiplication
    corr_xy = X_train.T @ Y_train

    # Select indices of the highest correlated predictors
    best = np.argsort(np.abs(corr_xy))[-H:]

    # Subset the training and validation data to include only these features
    X_train_mx = X_train[:, best]
    X_val_mx = X_val[:, best]

    # Train a K-Nearest Neighbors classifier
    knn = KNN(n_neighbors=1)
    knn.fit(X_train_mx, Y_train)

    # Evaluate the classifier on the validation set
    accuracy = knn.score(X_val_mx, Y_val)
    correct.append(accuracy)

# Calculate the average error rate
error_rate = (1 - np.mean(correct)) * 100

# Print the error rate
print(f'Percent error: {error_rate:.2f}%')

Percent error: 47.92%
